# Data Collection

This is the data collection section.

1. Trips and Station Data
   For Data Exploration
2. Full/Empty outage data from csv 
   May consider only including Spring-Fall to exclude Boston winter months
3. Bike/Dock availability data
   Pretty print json to extract 
4. Rainfall data: scrape me

In [5]:
# Import Libraries 

import os
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime

# Trip and Station Data

Hubway has published downloadable files of trip data for each month going back to 2015. According to their website (https://www.thehubway.com/system-data), the data has already been processed to remove trips that are taken by staff as they service the system and any potentially erroneous trips.

This study will focus only the months X-X in 2016. Many non-Cambridge docking stations are shut down for the winter months, so the spring to fall months have more comprehensive coverage.

The trip data was extracted from https://s3.amazonaws.com/hubway-data/index.html in .csv files broken out by months. I concatenated the monthly data for 2016 into a single pandas dataframe.

In [142]:
trips
# import "display" so you can do multiple displays
adam
from ipython.something import display

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
tripduration,,,,,,,,,,,,,,
1939,2016-01-01 00:08:07,2016-01-01 00:40:26,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,19,Buswell St. at Park Dr.,42.347241,-71.105301,1406,Subscriber,\N,0
1927,2016-01-01 00:08:19,2016-01-01 00:40:27,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,19,Buswell St. at Park Dr.,42.347241,-71.105301,1121,Subscriber,\N,0
1813,2016-01-01 00:10:15,2016-01-01 00:40:29,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,19,Buswell St. at Park Dr.,42.347241,-71.105301,474,Subscriber,\N,0
1788,2016-01-01 00:10:22,2016-01-01 00:40:11,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,19,Buswell St. at Park Dr.,42.347241,-71.105301,1504,Subscriber,\N,0
941,2016-01-01 00:15:36,2016-01-01 00:31:17,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,67,MIT at Mass Ave / Amherst St,42.358100,-71.093198,913,Customer,1990,1
323,2016-01-01 00:17:15,2016-01-01 00:22:39,110,Harvard University Gund Hall at Quincy St / Ki...,42.376369,-71.114025,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,-71.101427,620,Subscriber,1993,1
1072,2016-01-01 00:25:13,2016-01-01 00:43:06,107,Ames St at Main St,42.362500,-71.088220,176,Lesley University,42.386748,-71.119019,204,Subscriber,1984,2
452,2016-01-01 00:25:24,2016-01-01 00:32:57,141,Kendall Street,42.363560,-71.082168,90,Lechmere Station at Cambridge St / First St,42.370677,-71.076529,750,Subscriber,1965,1
285,2016-01-01 00:45:02,2016-01-01 00:49:47,178,MIT Pacific St at Purrington St,42.359573,-71.101295,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,909,Subscriber,\N,0


In [6]:
path = '/Users/lxu213/Documents/data/hubway/'
trips_path = path + 'Trips_2016/'

# list trip files by monthly data, excluding mac-file .DS_Store 
trip_files = os.listdir(trips_path)[1:]

def concat_trip_data(trips_path, trip_files):
    """ Concatenates all of the monthly hubway trip data of 2016 """
    trips = pd.DataFrame()
    for file in trip_files:
        add_trip = pd.DataFrame.from_csv(trips_path + file)
        trips = trips.append(add_trip)
    return trips

# trips = concat_trip_data(trips_path, trip_files)
# trips.to_csv('2016-tripdata.csv')

trips = pd.DataFrame.from_csv(trips_path + '2016-tripdata.csv')
print 'trips', trips.shape
stations = pd.DataFrame.from_csv(path + 'Stations_2016.csv')
print 'stations', stations.shape
stations.head()

OSError: [Errno 2] No such file or directory: '/Users/lxu213/Documents/data/hubway/Trips_2016/'

# Station Outage Data

An additional resource provided by the Hubway Challenge is the "Hubway Tracker," which provides real-time station, docking, and a bike availability tool. This tracker is used by Hubway’s Field Operations Managers for dispatch and station rebalancing efforts.

The station outage data was extracted from http://www.hubwaytracker.com/outage_history.php and available as a single downloadable file.

In [144]:
outages = pd.DataFrame.from_csv(path + 'Outages_2016.csv')
outages.head()


'hello'

# Bike and Dock Availability Data

The actual number of available bikes and docks is not available in an easily downloadable format from the Hubway Challenge. However, this data can be extracted as a json by making direct HTTP POST requests to http://www.hubwaytracker.com/getstation_json.php.


In [11]:
url = 'http://www.hubwaytracker.com/getstation_json.php'
payload = {'q': 39, 's': '2017-03-26', 'e': '2017-04-02', 
           'wd': 'true', 'we': 'true'}

r = requests.post(url, data=payload).json()

# print type(r)         # <type 'list'>
# print len(r)          # 2





# print len(r)
# print r[0]['label']
# print r[0].keys()
# print r[1]['data']
# print r[1]['label']


# json.dumps(parsed)

# pd.read_json(json.dumps(parsed))
# stuck on how to convert json tuples > pandas dataframe

AttributeError: 'list' object has no attribute 'text'

# Rainfall Data

Hubway bike usage is highly dependent on weather. The colder and wetter the day, the less inclined a user will be to hop on a bike! Although there are many open-source websites that provide weather and rainfall data, the industry standard for quality rainfall data comes from the National Oceanic and Atmospheric Administration's (formerly the National Climatic Data Center) Quality Controlled Local Climatological Data (QCLCD).

The rainfall data was extracted from https://www.ncdc.noaa.gov/qclcd/QCLCD.

In [116]:
url = 'https://www.ncdc.noaa.gov/qclcd/QCLCD'
payload = {'reqday': 'E', 'stnid': 'n/a', 'prior': 'N','qcname': 'VER2', 
           'VARVALUE': '14739201601', 'which': 'ASCII Download (Hourly Obs.) (10A)'}

r = requests.post(url, data=payload)
print type(r)
print type(r.text)
print r.text

#not a json... it's unicode 
parsed = json.load(r.text)

1. save out to a file and re-import (use with... as f...)
2. use string io library - it helps you make fake files in memory 
google pandas -> read under string csv string io

<class 'requests.models.Response'>
<type 'unicode'>
Hourly Obs
<pre>
Month/Year: 01/2016
Station Location: GEN E L LOGAN INTERNATIONAL AIRPORT (14739)
Lat: 42.360
Lon: -71.009
Elev: 12 ft. above sea level
WBAN,Date,Time,StationType,SkyCondition,SkyConditionFlag,Visibility,VisibilityFlag,WeatherType,WeatherTypeFlag,DryBulbFarenheit,DryBulbFarenheitFlag,DryBulbCelsius,DryBulbCelsiusFlag,WetBulbFarenheit,WetBulbFarenheitFlag,WetBulbCelsius,WetBulbCelsiusFlag,DewPointFarenheit,DewPointFarenheitFlag,DewPointCelsius,DewPointCelsiusFlag,RelativeHumidity,RelativeHumidityFlag,WindSpeed,WindSpeedFlag,WindDirection,WindDirectionFlag,ValueForWindCharacter,ValueForWindCharacterFlag,StationPressure,StationPressureFlag,PressureTendency,PressureTendencyFlag,PressureChange,PressureChangeFlag,SeaLevelPressure,SeaLevelPressureFlag,RecordType,RecordTypeFlag,HourlyPrecip,HourlyPrecipFlag,Altimeter,AltimeterFlag
14739,20160101,0054,11,OVC040, ,10.00, , , ,39, ,3.9, ,34, ,1.1, ,26, ,-3.3, , 60, ,11, ,250, ,2

AttributeError: 'unicode' object has no attribute 'read'

# Data Exploration 

Examine trip distribution by age, gender, time of day

In [118]:
%matplotlib inline

time = 

X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)

plt.plot(X, C)
plt.plot(X, S)

plt.show()

print trips.head()

                        starttime             stoptime  start station id  \
tripduration                                                               
1939          2016-01-01 00:08:07  2016-01-01 00:40:26                36   
1927          2016-01-01 00:08:19  2016-01-01 00:40:27                36   
1813          2016-01-01 00:10:15  2016-01-01 00:40:29                36   
1788          2016-01-01 00:10:22  2016-01-01 00:40:11                36   
941           2016-01-01 00:15:36  2016-01-01 00:31:17                36   

                                    start station name  \
tripduration                                             
1939          Boston Public Library - 700 Boylston St.   
1927          Boston Public Library - 700 Boylston St.   
1813          Boston Public Library - 700 Boylston St.   
1788          Boston Public Library - 700 Boylston St.   
941           Boston Public Library - 700 Boylston St.   

              start station latitude  start station longitud

In [139]:
#mdates.trips['starttime'][1:10]

# convert the epoch format to matplotlib date format 
#mpl_data = mdates.epoch2num(data)
print pd.to_datetime(trips['starttime'][1:10])
print datetime.replace(second=0)


# converted_dates = map(datetime.datetime.strptime, datelist, len(datelist)*['%Y-%m-%d'])
# x_axis = (converted_dates)
# formatter = dates.DateFormatter('%Y-%m-%d')


# plot it
# fig, ax = plt.subplots(1,1)
# ax.hist(x, color='lightblue')
# ax.xaxis.set_major_locator(x.YearLocator())
# ax.xaxis.set_major_formatter(x.DateFormatter('%d.%m.%y'))
# plt.show()

ImportError: No module named datetime